# Azure AI Foundry Agent Service - MCP Demo

This notebook demonstrates how to create and interact with an Azure AI Foundry Agent using the MCP Tool. Based on the sample from [here](https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-agents/samples/agents_tools/sample_agents_mcp.py). Supported regions at time of writing: `westus`, `westus2`, `uaenorth`, `southindia` and `switzerlandnorth`.

Before running the sample:

`pip install azure-ai-projects azure-ai-agents azure-identity --pre`

Set these environment variables with your own values:
1) `PROJECT_ENDPOINT2` - The Azure AI Project endpoint, as found in the Overview page of your Azure AI Foundry portal.
2) `MODEL_DEPLOYMENT_NAME` - The deployment name of the AI model, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) `MCP_SERVER_URL` - The URL of your MCP server endpoint.
4) `MCP_SERVER_LABEL` - A label for your MCP server.

In [1]:
import os
import time

from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    McpTool,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
)
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

# Load any values stored in a local .env file
load_dotenv()

# MCP server configuration
mcp_server_url  = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "MicrosoftDocs")


In [2]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT2"],  # ← ensure this env var is set
    credential=DefaultAzureCredential(),
)

# Convenience handle for the Agents sub‑client
agents_client = project_client.agents


In [3]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],          # Optional initial allow‑list
)


In [4]:
search_api_code = "microsoft_docs_search"
mcp_tool.allow_tool(search_api_code)
print(f"Allowed tools: {mcp_tool.allowed_tools}")

Allowed tools: ['microsoft_docs_search']


In [ ]:
agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],          # deployment name
    name="my-mcp-agent",
    instructions=(
        "You are a helpful agent that can use MCP tools to assist users. "
        "Use the available MCP tools to answer questions and perform tasks."
    ),
    tools=mcp_tool.definitions,
)

print(f"Created agent, ID: {agent.id}")
print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Failed to invoke the Azure CLI
	AzurePowerShellCredential: Az.Account module >= 2.2.0 is not installed
	AzureDeveloperCliCredential: Please run 'azd auth login' from a command prompt to authenticate before using this credential.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.

KeyboardInterrupt



: 

In [ ]:
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_aeD911R4CGQOlPwH5P3P3hbi


In [ ]:
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Does Azure AI Foundry Agent Service support Model Context Protocol as a tool?",
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_QVO8LntFi76pxJvF3A21swXv


In [ ]:
# Configure headers / approval mode (optional)
mcp_tool.update_headers("SuperSecret", "123456")

mcp_tool.set_approval_mode("never")          # Disable human approval

run = agents_client.runs.create(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_resources=mcp_tool.resources,
)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided – cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                print(f"Approving tool call: {tool_call}")
                tool_approvals.append(
                    ToolApproval(
                        tool_call_id=tool_call.id,
                        approve=True,
                        headers=mcp_tool.headers,
                    )
                )

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            agents_client.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Created run, ID: run_06nqNi5GiiLDcvwdbN9LGXzm
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


In [ ]:
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])
    if tool_calls:
        print("  MCP Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")
    print()     # blank line for readability

Step step_7DDhdIYH0FYFWQXKlAqmE8xT status: completed

Step step_rWQIutDHfHdzd2vGvOaxicwr status: completed
  MCP Tool calls:
    Tool Call ID: call_y06uBapoEZ71N49Zfesuabj6
    Type: mcp



In [ ]:
messages = agents_client.messages.list(thread_id=thread.id)

print("\nConversation")
print("-" * 50)
for msg in reversed(list(messages)):
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role.upper()}: {last_text.text.value}")
        print("-" * 50)


Conversation
--------------------------------------------------
USER: Does Azure AI Foundry Agent Service support Model Context Protocol as a tool?
--------------------------------------------------
ASSISTANT: Yes, Azure AI Foundry Agent Service supports Model Context Protocol (MCP) as a tool.

You can extend your agents by connecting them to tools hosted on remote Model Context Protocol (MCP) servers using the built-in "MCP tool". This allows agents to integrate with external services that expose MCP endpoints, letting you enhance the agent experience with third-party or custom tools in a standard, scalable way.

Key points from the official documentation:
- The MCP tool enables your agents to connect to any remote MCP server.
- MCP is an open standard for exposing tools and contextual data to LLMs and AI agents.
- To use it, you configure your agent with the MCP server endpoint, and may pass custom headers (such as authentication info) per each run.
- This capability is available as

In [ ]:
print("\nDemonstrating dynamic tool management")
print(f"Current allowed tools: {mcp_tool.allowed_tools}")

try:
    mcp_tool.disallow_tool(search_api_code)
    print(f"After removing {search_api_code}: {mcp_tool.allowed_tools}")
except ValueError as err:
    print(f"Error removing tool: {err}")


Demonstrating dynamic tool management
Current allowed tools: ['microsoft_docs_search']
After removing microsoft_docs_search: []


In [ ]:
agents_client.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
